# Lab1_Part3
# Keerthana Baskaran
# Esther Raja Kumari


# Loading all the Required Libraries

In [1]:
library(twitteR)
library(ROAuth)
library(ggplot2)
library(ggmap)
library(maptools)
consumer_key <- ""
consumer_secret <- ""
access_token <- ""
access_secret <- ""
register_google(key = "")

Warning message:
"package 'twitteR' was built under R version 3.5.3"Warning message:
"package 'ROAuth' was built under R version 3.5.3"Warning message:
"package 'ggplot2' was built under R version 3.5.3"Warning message:
"package 'ggmap' was built under R version 3.5.3"Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.
Please cite ggmap if you use it! See citation("ggmap") for details.
Warning message:
"package 'maptools' was built under R version 3.5.3"Loading required package: sp
Warning message:
"package 'sp' was built under R version 3.5.2"Checking rgeos availability: TRUE


# Searching for tweets and removing retweets

In [2]:
setup_twitter_oauth(consumer_key, consumer_secret, access_token, access_secret)
tweets_res <- searchTwitter("flu OR usflu OR usinfluenza OR influenza OR H3N2 OR H1N1 OR flutests OR positiveflutest OR fluhospitalizations OR flumortality OR UnitedStatesflu OR fluepidemic OR statesflu OR flushot OR flu2018 OR flu2017 OR flu 2019", n =20,lang="en",geocode="40.482405,-97.413745,2280mi")
tweets.df <- twListToDF(tweets_res)
tweets.df <- tweets.df[((tweets.df$isRetweet == FALSE)),]
twitter_users <- lookupUsers(users = tweets.df$screenName, includeNA = TRUE)
twitter_users.df <- twListToDF(twitter_users)
tweets.df$location <- twitter_users.df$location[match(tweets.df$screenName, twitter_users.df$screenName)]
tweets.df <- tweets.df[!(is.na(tweets.df$location) | (tweets.df$location == "")),]

[1] "Using direct authentication"


In [3]:
dim(tweets.df)

[1] 10 17

# Getting the states from latitude and longitude using revgeo

In [4]:
#Adding a column state values to store the information of the states which we get from the reverse geocode.
#I used geocode to get the information of the latitude and longitude through the user information.
library(revgeo)
tweets.df <- tweets.df[!(is.na(tweets.df$location) | (tweets.df$location == "")),]
namevector <- c("states_col")
tweets.df[ , namevector] <- NA
for(i in 1:nrow(tweets.df)) {
    
    tweet <- tweets.df[i,]
    add <- toString(tweet$location)
    tryCatch({ 
      result <- geocode(add,output = "more", source = "google")
      tweets.df$longitude[i] <- result$lon
      tweets.df$latitude[i] <- result$lat
    }, error=function(e){ print (e)})

  }
revresult<-revgeo(tweets.df$longitude, tweets.df$latitude, output = 'frame')
tweets.df$states_col<-revresult$state
old_data <-data.frame(read.csv("C://Users//Esther//Desktop//MS-CS-2nd sem//Data Intensive Computing//ESTHERRA_LAB1//Part3//FluTweetsData.csv")) 

Warning message:
"package 'revgeo' was built under R version 3.5.2"Source : https://maps.googleapis.com/maps/api/geocode/json?address=Arlington,+VA&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Durham,+NC&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=United+States&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Washington,+D.C.,+U.S.A.&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=New+York,+USA&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Texas&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Baltimore,+MD&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Baltimore,+MD&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Trinidad+and+Tobago&key=xxx
Source : https://maps.googleapis.com/maps/api/geocode/json?address=Cleveland&key=xxx


[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-77.0909809&lat=38.8816208"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-78.898619&lat=35.9940329"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-95.712891&lat=37.09024"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-77.0368707&lat=38.9071923"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-74.0059728&lat=40.7127753"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-99.9018131&lat=31.9685988"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-76.6121893&lat=39.2903848"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-76.6121893&lat=39.2903848"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-61.222503&lat=10.691803"
[1] "Getting geocode data from Photon: http://photon.komoot.de/reverse?lon=-81.

# Writing unique data into csv file

In [5]:
# Finding the unique data from the previous data and the data we have extraxted from the unique function.Updating the csv file everytime with the new unique data
final_data <- unique(rbind(old_data, tweets.df))
write.csv(final_data, "FluTweetsData.csv", row.names = F )
dim(final_data)

Warning message in `[<-.factor`(`*tmp*`, ri, value = structure(c(1554479642, 1554479508, :
"invalid factor level, NA generated"

[1] 3380   18